# MODEL 1:  Uninsured Women in Chicago (2017)
--------------------------------------------------------------------------------------------------------------------------------

This notebook goes through the steps of creating a model to predict the population of uninsured Chicago women in 2017. We follow a 2-step modeling process to estimate the percentage of women in each Census tract in Chicago that are likely to be uninsured. In the first step, we'll train a model on individual-level data, where each row is a person. In the second step, we'll train a model on geographic-level data, where we use the same features as the individual-level model, but the values are aggregated by our geographic-level of interest (in this case, Census tracts). We'll then use our model to predict the proportion of uninsured women in each Census tract in Chicago.

This 2-step modeling process helps us avoid a reverse ecological fallacy problem, or an exception fallacy, which is a potential problem that arises from modeling based off survey response data.

To conduct our analyses, we'll be using the Civis Analytics platform API to connect to our data, create tables, and query our data. Through the Civis API, we'll also be able to use CivisML, our machine learning package. We'll use this to train and test our models, as well as make predictions.

To learn more about Civis Analytics and understand the data science platform we use to build this model, check out our website at the following link: https://www.civisanalytics.com/

#### NOTE: Some variable names and functions have been changed to protect proprietary information.

In [ ]:
import civis

import pandas as pd
import numpy as np
from civis.ml import ModelPipeline  # we'll be using Civis's model pipeline to create and run our models

client = civis.APIClient()

--------------------------------------------------------------------------------------------------------------------------------

# STEP 1: Create Individual-Level Training Table (female-only)

1.  Grab data for modeling
2.  Append on 2016 uninsured scores 
3.  Append responses from the survey we ran in 2017 (Oct - Nov 2017) -- only has data for women
4.  Recode survey question about insurance status to be binary variable where uninsured = 1, insured = 0


In [ ]:
%%civisquery -- this allows us to query our data using SQL in this cell


SET SEED TO .42;

DROP VIEW IF EXISTS cdph_train_uninsured;
CREATE VIEW cdph_train_uninsured AS
SELECT *
FROM
(
    SELECT full_modeling_data.*, 
    s.cdph_uninsured,
    s.weight
    FROM
    (
        SELECT *
        FROM
        (
            SELECT 
            id,
            weight,
            DECODE(cdph_insured,
                   'Yes - through the government (Medicare, Medicaid)', 0,
                   'Yes - through my employer or spouse/partner\'s employer, or I purchase it myself', 0,
                   'No - I don\'t have health insurance', 1,
                   NULL) AS cdph_uninsured,
            ROW_NUMBER() OVER (PARTITION BY id) AS dupes -- remove duplicate survey respondents
            FROM cdph_survey
       ) WHERE dupes = 1
    ) AS s 
    LEFT JOIN
    (
      SELECT *
      FROM modeling_data AS md
      LEFT JOIN
      (
          SELECT join_key, uninsured_2016
          FROM uninsured2016_data
      ) AS 2016data
      ON md.id = 2016data.join_key
    ) AS full_modeling_data
    ON full_modeling_data.id = s.id
)
WHERE cdph_uninsured IS NOT NULL
;


After creating the training table, we'll check the distribution of the target variable (cdph_uninsured) to ensure that the target event is not too rare.

We see that while few people are uninsured, the event is not too rare to the point where we have to rebalance the training data.

In [ ]:
queries = ["select count(cdph_uninsured) from cdph_train_uninsured",
        "select count(cdph_uninsured) from cdph_train_uninsured where cdph_uninsured = 0",
        "select count(cdph_uninsured) from cdph_train_uninsured where cdph_uninsured = 1"
        ]

print("\n------------------ CHECKING TARGET VARIABLE COUNTS ------------------")
for query in queries:
    sql_q = query
    print(sql_q)
    q = client.queries.post(database="database_number",
                            sql= sql_q,
                            preview_rows=100
                           )
    
    while client.queries.get(q['id'])['state'] != "succeeded":
        pass
    print(client.queries.get(q['id'])['result_rows'])

--------------------------------------------------------------------------------------------------------------------------------

# STEP 2: Train Individual-level Model
To train our model, we'll first specify a few classifier models offered in the modeling package Civis uses, CivisML. 

In [ ]:
models = {
    'sparse_logistic': {},
    'extra_trees_classifier': "hyperband",
    'gradient_boosting_classifier': "hyperband",
    'random_forest_classifier': "hyperband"
} 

futures = []



Next, we'll loop through each of the models we specified. CivisML allows us to tune the hyperparameters of our models using "hyperband", which is a more efficient approach to hyperparameter optimization.

For each model, we'll create a pipeline where we provide the dependent variable and a primary key (i.e. a column with a unique indicator for each row or observation). We can also choose to exclude specific columns. We will then train the model using the training table we created.

The CivisML modeling pipeline automatically uses stratified k-fold cross validation to cross validate the model.

In [ ]:
for i, params in models.items():
        
    print("Currently testing model: " + i)
    print("Params: " + str(params))
    print("--------------------------")

    m = ModelPipeline(model = i, 
                      model_name = 'Uninsured Model ' + i + ' DV is: cdph_uninsured',
                      dependent_variable = 'cdph_uninsured',
                      primary_key = 'id',
                      excluded_columns = ['state', 
                                          'join_key',
                                          'cdph_pregnant',
                                          'cdph_pregnant_age',
                                          'cdph_diagnosis_breastcancer',
                                          'cdph_family_breastcancer',
                                          'cdph_brca_test',
                                          'cdph_oral_contraceptives',
                                          'cdph_female_hormones',
                                         ],
                      cross_validation_parameters = params,
                      memory_requested = 3000,
                      cpu_requested = 800)

    train = m.train(table_name = 'cdph_train_uninsured',
                    fit_params = {'sample_weight': 'weight'},
                    database_name = 'database')
    futures.append(train)
        


Check to see if the models are running in the Civis Platform:

In [ ]:
for f in futures:
    print("Model running?  " + str(f.running()))
    print("Job ID: " + str(f.job_id))
    print("Train Job ID: " + str(f.train_job_id))
    print("Run ID: " + str(f.train_run_id))
    print("------------------------------------------")

--------------------------------------------------------------------------------------------------------------------------------

# STEP 3: Compare Individual-Level Model Performance
After our models have finished running, we'll print out the metrics for each one and compare them. We'll select the best performing model to score our dataset. 

#### Among the models we tested, the extra trees classifier model ended up having the best performance.

In [ ]:
for f in futures: 
    print("\n************************************\n")
    if str(f.running()) == "False" and f.metadata['run']['status'] != "exception":
        print("MODEL: " + f.metadata['model']['model'])
        print("DV: " + f.metadata['run']['configuration']['data']['y'][0])
        print("TRAINING TABLE: " + f.metadata['data_platform']['table_source']['tablename'])
        try:
            print("\n-----------------------\n")
            print("AUC: " + str(f.metrics['roc_auc']))
            print("\n------------------------\n")
            print("CONFUSION MATRIX:  " + str(f.metrics['confusion_matrix']))
            print("\n------------------------------------\n")
            print("BEST PARAMS:")
            print(f.metadata['model']['cv_best_params'])
        except:
            pass
        print("\n************************************\n")
    else:
        print("Model not finished running")



--------------------------------------------------------------------------------------------------------------------------------

# STEP 4: Create Individual-Level Scoring Table and Score
We'll create a scoring table with the same features as our training set. This scoring table only has data on females.

In [ ]:
%%civisquery

DROP VIEW IF EXISTS score_table_uninsured2017;
CREATE VIEW score_table_uninsured2017 AS
SELECT A.*, B.score AS uninsured_2016
FROM
(
    (
        SELECT *
        FROM modeling_data
    ) AS A
    LEFT JOIN
    (
        SELECT score, join_key2
        FROM 2016scores
    ) AS B
    ON A.id = B.join_key2
)
WHERE B.score IS NOT NULL
;



Using the Civis API, we'll grab the model with the best performance (sparse logistic), and use this model to score our data set. 

In [ ]:
# Grab Job ID, Run ID, name from output or Civis Platform UI
job_id = "ID NUMBER"
run_id = client.jobs.get(job_id)['last_run']['id']
name = client.jobs.get(job_id)['name']

# Print Model Info
print("NAME: " + name)
print("JOB ID: " + str(job_id))
print("RUN ID: " + str(run_id))

# Load model
loaded_model = ModelPipeline.from_existing(job_id, run_id)

model_type = loaded_model.model
print(model_type)  # model type is sparse logistic

# Score table using model
scoring = loaded_model.predict(table_name = "score_table_uninsured2017", 
                        database_name = "database",
                        output_table = "uninsured2017")

--------------------------------------------------------------------------------------------------------------------------------

# STEP 5: Aggregate Scores by Geographic Level; Create Aggregate Training & Scoring Tables

Now, we'll create a new training table and scoring table, where the features are aggregated by the geographic level of interest (i.e. Census tract). However, for our training table our survey response data will still be used as the dependent variable. 

We'll also use the scores from our individual-level as a feature in our geographic-level training and scoring tables. We'll take the average scores for each geographic level (i.e. each Census tract), and then we'll append them onto our training and scoring tables. 




### Aggregate features at the Census tract level

In [ ]:
%%civisquery 

-- Dependent views --
DROP VIEW IF EXISTS cdph_train_uninsured_all_agg;
DROP VIEW IF EXISTS agg_score_table_uninsured2017;


-- Create aggregated table by census tract --
DROP VIEW IF EXISTS tract_aggregate_basefile;
CREATE VIEW tract_aggregate_basefile AS
SELECT
  LEFT(census_block, 11) AS census_tract -- ID Census tracts
  , AVG(head_hh::float) AS head_hh  
  , AVG(hh_count::float) AS hh_count  
  , AVG(age::float) AS age  
  , AVG(age_18_34::float) AS age_18_34  
  , AVG(age_35_49::float) AS age_35_49 
  , AVG(age_50_64::float) AS age_50_64 
  , AVG(age_65_plus::float) AS age_65_plus 
  , AVG(hh_avg_age::float) AS hh_avg_age  
  , AVG(race_afam::float) AS race_afam
  , AVG(race_hispanic::float) AS race_hispanic 
  , AVG(race_asian::float) AS race_asian 
  , AVG(race_white::float) AS race_white 
  , AVG(race_native::float) AS race_native  
  , AVG(hh_race_afam::float) AS hh_race_afam  
  , AVG(hh_race_hispanic::float) AS hh_race_hispanic  
  , AVG(hh_race_asian::float) AS hh_race_asian  
  , AVG(hh_race_white::float) AS hh_race_white  
  , AVG(hh_race_native::float) AS hh_race_native  
  , AVG(hh_all_afam::float) AS hh_all_afam  
  , AVG(hh_all_hispanic::float) AS hh_all_hispanic  
  , AVG(hh_all_asian::float) AS hh_all_asian  
  , AVG(hh_all_white::float) AS hh_all_white  
  , AVG(hh_all_native::float) AS hh_all_native  
  , AVG(subeth_african_american::float) AS subeth_african_american  
  , AVG(subeth_west_indian::float) AS subeth_west_indian  
  , AVG(subeth_haitian::float) AS subeth_haitian  
  , AVG(subeth_african::float) AS subeth_african  
  , AVG(subeth_other_black::float) AS subeth_other_black  
  , AVG(subeth_mexican::float) AS subeth_mexican  
  , AVG(subeth_cuban::float) AS subeth_cuban  
  , AVG(subeth_puerto_rican::float) AS subeth_puerto_rican  
  , AVG(subeth_dominican::float) AS subeth_dominican  
  , AVG(subeth_other_latin_american::float) AS subeth_other_latin_american  
  , AVG(subeth_other_hispanic::float) AS subeth_other_hispanic  
  , AVG(subeth_chinese::float) AS subeth_chinese  
  , AVG(subeth_indian::float) AS subeth_indian  
  , AVG(subeth_filipino::float) AS subeth_filipino  
  , AVG(subeth_japanese::float) AS subeth_japanese  
  , AVG(subeth_vietnamese::float) AS subeth_vietnamese  
  , AVG(subeth_korean::float) AS subeth_korean  
  , AVG(subeth_other_asian::float) AS subeth_other_asian  
  , AVG(subeth_hmong::float) AS subeth_hmong  
  , AVG(spanish::float) AS spanish  
  , AVG(gender::float) AS gender  
  , AVG(hh_gender::float) AS hh_gender  
  , AVG(vb_phone_presence::float) AS vb_phone_presence  
  , AVG(vb_phone_listed::float) AS vb_phone_listed  
  , AVG(vb_phone_unlisted::float) AS vb_phone_unlisted  
  , AVG(vb_email_append_ind::float) AS vb_email_append_ind  
  , AVG(vb_email_append_hh::float) AS vb_email_append_hh  
  , AVG(vb_deceased::float) AS vb_deceased  
  , AVG(elections_pres_2012::float) AS electionses_2012  
  , AVG(elections_pres_2008::float) AS electionses_2008  
  , AVG(elections_pres_2004::float) AS electionses_2004  
  , AVG(elections_gov_recent::float) AS elections_gov_recent  
  , AVG(elections_state_avg::float) AS elections_state_avg  
  , AVG(elections_fedstate_diff::float) AS elections_fedstate_diff  
  , AVG(married::float) AS married  
  , AVG(unmarried::float) AS unmarried  
  , AVG(children_in_hh::float) AS children_in_hh  
  , AVG(num_children_in_hh::float) AS num_children_in_hh  
  , AVG(young_children_in_hh::float) AS young_children_in_hh  
  , AVG(teenage_children_in_hh::float) AS teenage_children_in_hh  
  , AVG(education_highschool::float) AS education_highschool  
  , AVG(education_collegegrad::float) AS education_collegegrad  
  , AVG(num_in_hh::float) AS num_in_hh  
  , AVG(religion_jewish::float) AS religion_jewish  
  , AVG(religion_mormon::float) AS religion_mormon  
  , AVG(religion_muslim::float) AS religion_muslim  
  , AVG(religion_catholic::float) AS religion_catholic  
  , AVG(religion_evangelicalotestant::float) AS religion_evangelicalotestant  
  , AVG(religion_mainlineotestant::float) AS religion_mainlineotestant  
  , AVG(religion_orthodox_christian::float) AS religion_orthodox_christian  
  , AVG(religion_jehovah_witness::float) AS religion_jehovah_witness  
  , AVG(religion_hindu::float) AS religion_hindu  
  , AVG(religion_buddhist::float) AS religion_buddhist  
  , AVG(vehicleowner::float) AS vehicleowner  
  , AVG(homeowner::float) AS homeowner  
  , AVG(property_type_singlefamily::float) AS property_type_singlefamily  
  , AVG(property_type_multifamily::float) AS property_type_multifamily  
  , AVG(address_type_firm::float) AS address_type_firm  
  , AVG(address_type_general_delivery::float) AS address_type_general_delivery  
  , AVG(address_type_high_rise::float) AS address_type_high_rise  
  , AVG(address_type_po_box::float) AS address_type_po_box  
  , AVG(address_type_rural::float) AS address_type_rural  
  , AVG(address_type_street::float) AS address_type_street  
  , AVG(has_ncoa_return_code::float) AS has_ncoa_return_code  
  , AVG(length_of_residence::float) AS length_of_residence  
  , AVG(length_of_residence_is_null::float) AS length_of_residence_is_null  
  , AVG(home_sqft::float) AS home_sqft  
  , AVG(home_sqft_is_null::float) AS home_sqft_is_null  
  , AVG(home_total_value::float) AS home_total_value  
  , AVG(home_total_value_is_unknown::float) AS home_total_value_is_unknown  
  , AVG(log_home_value_bucket::float) AS log_home_value_bucket  
  , AVG(home_value_is_null::float) AS home_value_is_null  
  , AVG(home_purchase_year::float) AS home_purchase_year  
  , AVG(home_purchase_year_is_null::float) AS home_purchase_year_is_null  
  , AVG(log_household_net_worth_bucket::float) AS log_household_net_worth_bucket  
  , AVG(household_net_worth_is_null::float) AS household_net_worth_is_null  
  , AVG(head_hh_salary_amt::float) AS head_hh_salary_amt  
  , AVG(head_hh_salary_amt_is_null::float) AS head_hh_salary_amt_is_null  
  , AVG(log_household_income_bucket::float) AS log_household_income_bucket  
  , AVG(household_income_is_null::float) AS household_income_is_null  
  , AVG(donor_charity::float) AS donor_charity  
  , AVG(donor_enviro_causes::float) AS donor_enviro_causes  
  , AVG(hh_donor_charity_pctile::float) AS hh_donor_charity_pctile  
  , AVG(hh_donor_charity::float) AS hh_donor_charity  
  , AVG(hh_donor_enviro_causes::float) AS hh_donor_enviro_causes  
  , AVG(dog_enthusiast::float) AS dog_enthusiast  
  , AVG(cat_enthusiast::float) AS cat_enthusiast  
  , AVG(pet_enthusiast::float) AS pet_enthusiast  
  , AVG(travel_domestic_foreign::float) AS travel_domestic_foreign  
  , AVG(license_hunt_fish::float) AS license_hunt_fish  
  , AVG(religious_purchase::float) AS religious_purchase  
  , AVG(political_purchase::float) AS political_purchase  
  , AVG(health_institution_purchase::float) AS health_institution_purchase  
  , AVG(general_purchase::float) AS general_purchase  
  , AVG(hh_license_hunt_fish::float) AS hh_license_hunt_fish  
  , AVG(hh_religious_purchase::float) AS hh_religious_purchase  
  , AVG(hh_political_purchase::float) AS hh_political_purchase  
  , AVG(hh_health_institution_purchase::float) AS hh_health_institution_purchase  
  , AVG(hh_general_purchase::float) AS hh_general_purchase  
  , AVG(presence_of_cell_phone::float) AS presence_of_cell_phone  
  , AVG(presence_of_cell_phone_modeled::float) AS presence_of_cell_phone_modeled  
  , AVG(hh_has_credit_card::float) AS hh_has_credit_card  
  , AVG(online_is_online::float) AS online_is_online  
  , AVG(online_facebook::float) AS online_facebook  
  , AVG(online_purchaser::float) AS online_purchaser  
  , AVG(hh_online_social_network_twentile::float) AS hh_online_social_network_twentile  
  , AVG(hh_online_is_online::float) AS hh_online_is_online  
  , AVG(hh_online_facebook::float) AS hh_online_facebook  
  , AVG(employed::float) AS employed  
  , AVG(employed_unknown::float) AS employed_unknown  
  , AVG(retired::float) AS retired  
  , AVG(hh_business_owner::float) AS hh_business_owner  
  , AVG(emp_nurse::float) AS emp_nurse  
  , AVG(emp_beauty::float) AS emp_beauty  
  , AVG(emp_self_employed::float) AS emp_self_employed  
  , AVG(emp_healthcare::float) AS emp_healthcare  
  , AVG(emp_nursingovider::float) AS emp_nursingovider  
  , AVG(emp_dentalovider::float) AS emp_dentalovider  
  , AVG(emp_healthcareovider::float) AS emp_healthcareovider  
  , AVG(emp_realestate::float) AS emp_realestate  
  , AVG(emp_educator::float) AS emp_educator  
  , AVG(emp_pilot::float) AS emp_pilot  
  , AVG(emp_aviation_industry::float) AS emp_aviation_industry  
  , AVG(emp_postal::float) AS emp_postal  
  , AVG(emp_federal::float) AS emp_federal  
  , AVG(emp_military::float) AS emp_military  
  , AVG(hh_emp_active_military::float) AS hh_emp_active_military  
  , AVG(hh_emp_active_military_modeled::float) AS hh_emp_active_military_modeled  
  , AVG(hh_emp_inactive_military::float) AS hh_emp_inactive_military  
  , AVG(hh_emp_inactive_military_modeled::float) AS hh_emp_inactive_military_modeled  
  , AVG(hh_employed::float) AS hh_employed  
  , AVG(hh_retired::float) AS hh_retired  
  , AVG(hh_emp_nurse::float) AS hh_emp_nurse  
  , AVG(hh_emp_beauty::float) AS hh_emp_beauty  
  , AVG(hh_emp_healthcare::float) AS hh_emp_healthcare  
  , AVG(hh_emp_realestate::float) AS hh_emp_realestate  
  , AVG(hh_emp_educator::float) AS hh_emp_educator  
  , AVG(zip5_pct_catholic::float) AS zip5_pct_catholic  
  , AVG(zip5_pct_jewish::float) AS zip5_pct_jewish  
  , AVG(zip5_pct_dems_per_reg::float) AS zip5_pct_dems_per_reg  
  , AVG(zip5_pct_registered::float) AS zip5_pct_registered  
  , AVG(zip5_pct_feccontributions_dem_2way::float) AS zip5_pct_feccontributions_dem_2way  
  , AVG(urban::float) AS urban  
  , AVG(suburban::float) AS suburban  
  , AVG(rural::float) AS rural  
  , AVG(county_is_in_msa::float) AS county_is_in_msa  
  , AVG(is_in_place::float) AS is_in_place  
  , AVG(place_is_in_principal_city::float) AS place_is_in_principal_city  
  , AVG(pct_under18::float) AS pct_under18  
  , AVG(pct_18plus::float) AS pct_18plus  
  , AVG(pct_race5way_hispanic::float) AS pct_race5way_hispanic  
  , AVG(pct_race5way_black::float) AS pct_race5way_black  
  , AVG(pct_race5way_asian::float) AS pct_race5way_asian  
  , AVG(pct_race5way_native::float) AS pct_race5way_native  
  , AVG(total_hh::float) AS total_hh  
  , AVG(pct_hh_owned_with_loan::float) AS pct_hh_owned_with_loan  
  , AVG(pct_hh_owned_no_loan::float) AS pct_hh_owned_no_loan  
  , AVG(pct_hh_renter::float) AS pct_hh_renter  
  , AVG(pct_hh_single_family_head::float) AS pct_hh_single_family_head  
  , AVG(pct_hh_with_own_children_under18::float) AS pct_hh_with_own_children_under18  
  , AVG(pct_hh_1_person::float) AS pct_hh_1_person  
  , AVG(pct_hh_husband_wife_family::float) AS pct_hh_husband_wife_family  
  , AVG(pums11model_fpl_under138::float) AS pums11model_fpl_under138  
  , AVG(pums11model_fpl_139to400::float) AS pums11model_fpl_139to400  
  , AVG(civismodel_incomeunder40k::float) AS civismodel_incomeunder40k  
  , AVG(civismodel_incomeover80k::float) AS civismodel_incomeover80k  
  , AVG(total_pop::float) AS total_pop  
  , AVG(log_pop_density::float) AS log_pop_density  
  , AVG(log_total_hispanic_density::float) AS log_total_hispanic_density  
  , AVG(log_total_white_density::float) AS log_total_white_density  
  , AVG(log_total_black_density::float) AS log_total_black_density  
  , AVG(log_total_native_american_density::float) AS log_total_native_american_density  
  , AVG(log_total_asian_density::float) AS log_total_asian_density  
  , AVG(total_hawaiian_pac_islander_pct::float) AS total_hawaiian_pac_islander_pct  
  , AVG(log_total_other_race_density::float) AS log_total_other_race_density  
  , AVG(total_multi_race_pct::float) AS total_multi_race_pct  
  , AVG(log_total_multi_race_density::float) AS log_total_multi_race_density  
  , AVG(adult_pct::float) AS adult_pct  
  , AVG(adult_hispanic_pct::float) AS adult_hispanic_pct  
  , AVG(log_adult_hispanic_density::float) AS log_adult_hispanic_density  
  , AVG(adult_white_pct::float) AS adult_white_pct  
  , AVG(log_adult_white_density::float) AS log_adult_white_density  
  , AVG(adult_black_pct::float) AS adult_black_pct  
  , AVG(log_adult_black_density::float) AS log_adult_black_density  
  , AVG(adult_native_american_pct::float) AS adult_native_american_pct  
  , AVG(log_adult_native_american_density::float) AS log_adult_native_american_density  
  , AVG(adult_asian_pct::float) AS adult_asian_pct  
  , AVG(log_adult_asian_density::float) AS log_adult_asian_density  
  , AVG(adult_hawaiian_pac_islander_pct::float) AS adult_hawaiian_pac_islander_pct  
  , AVG(adult_other_race_pct::float) AS adult_other_race_pct  
  , AVG(log_adult_other_race_density::float) AS log_adult_other_race_density  
  , AVG(adult_multi_race_pct::float) AS adult_multi_race_pct  
  , AVG(log_adult_multi_race_density::float) AS log_adult_multi_race_density  
  , AVG(minor_pct::float) AS minor_pct  
  , AVG(minor_hispanic_pct::float) AS minor_hispanic_pct  
  , AVG(log_minor_hispanic_density::float) AS log_minor_hispanic_density  
  , AVG(minor_white_pct::float) AS minor_white_pct  
  , AVG(log_minor_white_density::float) AS log_minor_white_density  
  , AVG(minor_black_pct::float) AS minor_black_pct  
  , AVG(log_minor_black_density::float) AS log_minor_black_density  
  , AVG(minor_native_american_pct::float) AS minor_native_american_pct  
  , AVG(minor_asian_pct::float) AS minor_asian_pct  
  , AVG(log_minor_multi_race_density::float) AS log_minor_multi_race_density  
  , AVG(median_age::float) AS median_age  
  , AVG(median_age_male::float) AS median_age_male  
  , AVG(median_age_female::float) AS median_age_female  
  , AVG(univacant_density::float) AS univacant_density  
  , AVG(univacant_rented_pct::float) AS univacant_rented_pct  
  , AVG(univacant_for_sale_pct::float) AS univacant_for_sale_pct  
  , AVG(univacant_sold_pct::float) AS univacant_sold_pct  
  , AVG(univacant_seasonal_etc_use_pct::float) AS univacant_seasonal_etc_use_pct  
  , AVG(univacant_migrant_workers_pct::float) AS univacant_migrant_workers_pct  
  , AVG(univacant_other_pct::float) AS univacant_other_pct  
  , AVG(pct_employment_construction::float) AS pct_employment_construction  
  , AVG(pct_employment_manufacturing::float) AS pct_employment_manufacturing  
  , AVG(pct_employment_wholesale::float) AS pct_employment_wholesale  
  , AVG(pct_employment_retail::float) AS pct_employment_retail  
  , AVG(pct_employment_finance::float) AS pct_employment_finance  
  , AVG(pct_employment_management::float) AS pct_employment_management  
  , AVG(pct_employment_educational::float) AS pct_employment_educational  
  , AVG(pct_employment_health::float) AS pct_employment_health  
  , AVG(pct_employment_arts::float) AS pct_employment_arts  
  , AVG(indian_reservation::float) AS indian_reservation  
  , AVG(military_base::float) AS military_base  
  , AVG(pct_pop_corrections_facility::float) AS pct_pop_corrections_facility  
  , AVG(pct_pop_nursing_facility::float) AS pct_pop_nursing_facility  
  , AVG(pct_pop_military_quarters::float) AS pct_pop_military_quarters  
  , AVG(pct_pop_student_housing::float) AS pct_pop_student_housing  
  , AVG(pct_pop_evangelical::float) AS pct_pop_evangelical  
  , AVG(pct_pop_catholic::float) AS pct_pop_catholic  
  , AVG(pct_pop_blackotestant::float) AS pct_pop_blackotestant  
  , AVG(pct_pop_muslim::float) AS pct_pop_muslim  
  , AVG(pct_pop_mormon::float) AS pct_pop_mormon  
  , AVG(pct_pop_jewish::float) AS pct_pop_jewish  
  , AVG(pct_pop_buddhist::float) AS pct_pop_buddhist  
  , AVG(pct_pop_hindu::float) AS pct_pop_hindu  
  , AVG(median_age_white_male::float) AS median_age_white_male  
  , AVG(pct_white_male_less_high_school::float) AS pct_white_male_less_high_school  
  , AVG(pct_white_female_less_high_school::float) AS pct_white_female_less_high_school  
  , AVG(median_hh_income::float) AS median_hh_income  
  , AVG(pct_noncitizen::float) AS pct_noncitizen  
  , AVG(pct_moved_past_year_within_county::float) AS pct_moved_past_year_within_county  
  , AVG(pct_moved_past_year_all::float) AS pct_moved_past_year_all  
  , AVG(pct_leave_for_work_4pmto5am::float) AS pct_leave_for_work_4pmto5am  
  , AVG(pct_commute_over90min::float) AS pct_commute_over90min  
  , AVG(pct_vehicle_available::float) AS pct_vehicle_available  
  , AVG(pct_enrolled_in_higher_ed::float) AS pct_enrolled_in_higher_ed  
  , AVG(pct_educ_no_hs::float) AS pct_educ_no_hs  
  , AVG(pct_educ_bachelors::float) AS pct_educ_bachelors  
  , AVG(pct_in_labor_force::float) AS pct_in_labor_force  
  , AVG(pct_disabled::float) AS pct_disabled  
  , AVG(outflow_same_state::float) AS outflow_same_state  
  , AVG(outflow_foreign::float) AS outflow_foreign  
  , AVG(outflow_total::float) AS outflow_total  
  , AVG(inflow_same_state::float) AS inflow_same_state  
  , AVG(inflow_diff_state::float) AS inflow_diff_state  
  , AVG(inflow_foreign::float) AS inflow_foreign  
  , AVG(inflow_total::float) AS inflow_total  
  , AVG(exemptions_per_return::float) AS exemptions_per_return  
  , AVG(dependenper_return::float) AS dependenper_return  
  , AVG(pct_single_returns::float) AS pct_single_returns  
  , AVG(pct_joint_returns::float) AS pct_joint_returns  
  , AVG(pct_headofhousehold_returns::float) AS pct_headofhousehold_returns  
  , AVG(avg_adjusted_gross_income::float) AS avg_adjusted_gross_income  
  , AVG(pct_farm_returns::float) AS pct_farm_returns  
  , AVG(pct_retiree_returns::float) AS pct_retiree_returns  
  , AVG(pct_salarywages_returns::float) AS pct_salarywages_returns  
  , AVG(avg_amnt_salarywages::float) AS avg_amnt_salarywages  
  , AVG(pct_businessincome_returns::float) AS pct_businessincome_returns  
  , AVG(avg_amnt_businessincome::float) AS avg_amnt_businessincome  
  , AVG(avg_amnt_unemployed_benefits::float) AS avg_amnt_unemployed_benefits  
  , AVG(pct_socsecurity_returns::float) AS pct_socsecurity_returns  
  , AVG(avg_amnt_socsecurity_benefits::float) AS avg_amnt_socsecurity_benefits  
  , AVG(avg_amnt_contributions::float) AS avg_amnt_contributions  
  , AVG(pct_energy_taxcredit_returns::float) AS pct_energy_taxcredit_returns  
  , AVG(avg_amnt_energy_taxcredit_per_return::float) AS avg_amnt_energy_taxcredit_per_return  
  , AVG(avg_amnt_child_taxcredit_per_return::float) AS avg_amnt_child_taxcredit_per_return  
  , AVG(pct_eitc_returns::float) AS pct_eitc_returns  
  , AVG(avg_amnt_eitc_per_return::float) AS avg_amnt_eitc_per_return  
  , AVG(veteran_pct::float) AS veteran_pct  
  , AVG(employ_current_rate::float) AS employ_current_rate  
  , AVG(employ_ten_month_low::float) AS employ_ten_month_low  
  , AVG(retired_workers::float) AS retired_workers  
  , AVG(disabled_workers::float) AS disabled_workers  
  , AVG(widowers::float) AS widowers  
  , AVG(children::float) AS children  
  , AVG(retired_workers_benefits::float) AS retired_workers_benefits  
  , AVG(widower_benefits::float) AS widower_benefits  
  , AVG(total_crimes::float) AS total_crimes  
  , AVG(rape::float) AS rape  
  , AVG(agg_assault::float) AS agg_assault  
  , AVG(prostitution::float) AS prostitution  
  , AVG(sex_offense::float) AS sex_offense  
  , AVG(vehicle_theft::float) AS vehicle_theft  
  , AVG(disorderly_conduct::float) AS disorderly_conduct  
  , AVG(drug_total::float) AS drug_total  
  , AVG(median_home_list_price::float) AS median_home_list_price  
  , AVG(median_rental_price::float) AS median_rental_price  
  , AVG(median_listing_price_sqfoot::float) AS median_listing_price_sqfoot  
  , AVG(pct_homes_price_reduction::float) AS pct_homes_price_reduction  
  , AVG(median_price_reduction::float) AS median_price_reduction  
  , AVG(log_distance_to_starbucks::float) AS log_distance_to_starbucks  
  , AVG(log_distance_to_elementary_school::float) AS log_distance_to_elementary_school  
  , AVG(log_distance_to_high_school::float) AS log_distance_to_high_school  
  , AVG(log_distance_to_homeless_shelter::float) AS log_distance_to_homeless_shelter  
  , AVG(log_distance_to_hospital::float) AS log_distance_to_hospital  
  , AVG(log_distance_to_assisted_living::float) AS log_distance_to_assisted_living  
  , AVG(log_distance_to_independent_living::float) AS log_distance_to_independent_living  
  , AVG(log_distance_to_kindergarten::float) AS log_distance_to_kindergarten  
  , AVG(log_distance_to_middle_school::float) AS log_distance_to_middle_school  
  , AVG(log_distance_to_nursing_home::float) AS log_distance_to_nursing_home  
  , AVG(log_distance_to_synagogue::float) AS log_distance_to_synagogue  
  , AVG(log_distance_to_university::float) AS log_distance_to_university  
  , AVG(housing_temp_address::float) AS housing_temp_address  
  , AVG(amenities99_amenity_rank::float) AS amenities99_amenity_rank  
  , AVG(fast_food_restaurants::float) AS fast_food_restaurants  
  , AVG(full_service_restaurants::float) AS full_service_restaurants  
  , AVG(grocery_stores_per_cap::float) AS grocery_stores_per_cap  
  , AVG(rec_facilities::float) AS rec_facilities  
  , AVG(snap_benefiper_cap::float) AS snap_benefiper_cap  
  , AVG(wic_redemptions_per_cap::float) AS wic_redemptions_per_cap  
  , AVG(sfree_lunch_eligible_pct::float) AS sfree_lunch_eligible_pct  
  , AVG(sreduced_lunch_eligible_pct::float) AS sreduced_lunch_eligible_pct  
  , AVG(low_access_to_store_pct::float) AS low_access_to_store_pct  
  , AVG(no_car_low_access_to_store_pct::float) AS no_car_low_access_to_store_pct  
  , AVG(mail_return_pct::float) AS mail_return_pct  
  , AVG(parent::float) AS parent  
  , AVG(uninsured_2015::float) AS uninsured_2015  
  , AVG(marriage::float) AS marriage 
  , AVG(uninsured_2016) AS uninsured_2016
  FROM score_table_uninsured2017  -- use individual-level scoring table for model trained on individual-level data
  WHERE census_block IS NOT NULL
  GROUP BY 1   -- group by Census tracts
;

### Aggregate individual-level model scores at the Census tract level

In [ ]:
%%civisquery 

DROP VIEW IF EXISTS cdph_train_uninsured_all_agg;
DROP VIEW IF EXISTS agg_score_table_uninsured2017;
DROP VIEW IF EXISTS agg_uninsured2017_scores;
CREATE VIEW agg_uninsured2017_scores AS
SELECT 
LEFT(modeling.census_block, 11) AS census_tract
, AVG(scores.cdph_uninsured) AS uninsured2017_scores
FROM uninsured2017 AS scores
LEFT JOIN modeling_data AS modeling
ON modeling.id = scores.id
GROUP BY 1
;

### Create new scoring table with data aggregated at the census tract level; aggregated individual-level scores appended on as a column.
Each row is a geographic level.

In [ ]:
%%civisquery 

DROP VIEW IF EXISTS cdph_train_uninsured_all_agg;
DROP VIEW IF EXISTS agg_score_table_uninsured2017;
CREATE VIEW agg_score_table_uninsured2017 AS
SELECT 
bridge.ca,
agg_basefile.*,
agg_scores.uninsured2017_scores
FROM tract_aggregate_basefile AS agg_basefile
LEFT JOIN
agg_uninsured2017_scores AS agg_scores
ON agg_basefile.census_tract = agg_scores.census_tract
LEFT JOIN
ca_tract_bridge AS bridge   -- this is a table connect Census tracts to Chicago community areas
ON bridge.id = agg_basefile.census_tract
WHERE bridge.id IS NOT NULL  -- subsetting scoring table to only include Chicago Census tracts
;

### Create new training table with aggregated data; this is at the individual level but the features are aggregated by Census tract; aggregated individual-level scores are appended on as a column

Each row is an individual, but the features/columns are aggregated by geographic level.

In [ ]:
%%civisquery

DROP VIEW IF EXISTS scratch.cdph_train_uninsured_all_agg;
CREATE VIEW scratch.cdph_train_uninsured_all_agg AS
SELECT 
A.id,
A.cdph_uninsured,
A.weight,
B.*
FROM
(
    SELECT 
    train.id,
    train.cdph_uninsured,
    train.weight,
    LEFT(modeling.census_block, 11) AS census_tract
    FROM cdph_train_uninsured_all_female AS train
    LEFT JOIN modeling_data AS modeling
    ON train.id = modeling.id
) AS A
LEFT JOIN
(
    SELECT 
    agg_basefile.*, 
    agg_scores.uninsured2017_scores
    FROM tract_aggregate_basefile AS agg_basefile
    LEFT JOIN
    agg_uninsured2017_scores AS agg_scores
    ON agg_basefile.census_tract = agg_scores.census_tract
) AS B
ON A.census_tract = B.census_tract
WHERE A.cdph_uninsured IS NOT NULL
;

--------------------------------------------------------------------------------

# STEP 6: Train Geographic-Level Model
Set up modeling pipeline by specifying which models to train, and selecting the parameters for the different training tables.

In [ ]:
agg_models = {
    'sparse_logistic': {},
    'extra_trees_classifier': 'hyperband',
    'gradient_boosting_classifier': 'hyperband',
    'random_forest_classifier': 'hyperband'
} 


agg_futures = []

In [ ]:
for i, params in agg_models.items():

    print("Currently testing model: " + i)
    print("Params: " + str(params))
    print("--------------------------")

    m = ModelPipeline(model = i, 
                      model_name = 'AGG DATA, WEIGHTED -- ' + i + ' DV is cdph_uninsured',
                      dependent_variable = 'cdph_uninsured',
                      primary_key = 'id',
                      excluded_columns = ['census_tract'
                                         ],
                      cross_validation_parameters = params,
                      memory_requested = 5000,
                      cpu_requested = 1024)

    train = m.train(table_name = 'cdph_train_uninsured_all_agg',
                    fit_params = {'sample_weight': 'weight'},
                    database_name = 'database'
                    )
    agg_futures.append(train)

Check that jobs are running in the Civis platform:

In [ ]:
for f in agg_futures:
    print("Model running?  " + str(f.running()))
    print("Job ID: " + str(f.job_id))
    print("Train Job ID: " + str(f.train_job_id))
    print("Run ID: " + str(f.train_run_id))
    print("------------------------------------------")

-------------------------------------------------------------------------------------------

# STEP 7: Compare Geographic-level Model Performance

After our models have finished running, we'll print out the metrics for each one and compare them. We'll select the best performing model to score our dataset.

#### Among the models we tested, the sparse logistic model ended up having the best performance.

In [ ]:
for f in agg_futures: 
    print("\n************************************\n")
    if str(f.running()) == "False" and f.metadata['run']['status'] != "exception":
        print("MODEL: " + f.metadata['model']['model'])
        print("DV: " + f.metadata['run']['configuration']['data']['y'][0])
        print("TRAINING TABLE: " + f.metadata['data_platform']['table_source']['tablename'])
        print("Job ID: " + str(f.job_id))
        try:
            print("\n-----------------------\n")
            print("AUC: " + str(f.metrics['roc_auc']))
            print("\n------------------------\n")
            print("CONFUSION MATRIX:  " + str(f.metrics['confusion_matrix']))
            print("\n------------------------------------\n")
            print("BEST PARAMS:")
            print(f.metadata['model']['cv_best_params'])
        except:
            pass
        print("\n************************************\n")
    else:
        print("Model not finished running")

------------------------------------------------------------------------------

# STEP 8: Score Geographic-Level Scoring Table

### Load model trained on aggregated data for scoring

In [ ]:
job_id = "ID NUMBER"


run_id = client.jobs.get(job_id)['last_run']['id']
name = client.jobs.get(job_id)['name']

# Print Model Info
print("NAME: " + name)
print("JOB ID: " + str(job_id))
print("RUN ID: " + str(run_id))

# Load model
loaded_agg_model = ModelPipeline.from_existing(job_id, run_id)

model_type = loaded_agg_model.model
print(model_type)

### Score scoring table with aggregated data

In [ ]:
# Score table using model
scoring = loaded_agg_model.predict(table_name = "agg_score_table_uninsured2017", 
                               database_name = "database",
                               output_table = "uninsured2017_aggscores",
                               if_exists = "drop",
                               primary_key = "census_tract"
                              )